In [2]:
push!( LOAD_PATH, "." ); # folder que continen los modulos. Correr desde ahí dentro.
#
using Z_auxiliaresBRW, Z_auxiliaresGEN
using HDF5

In [3]:
arxname="/home/karel/BRWFiles/Isabel2019/control_02.brw"

"/home/karel/BRWFiles/Isabel2019/control_02.brw"

In [6]:
pmem=filesize(arxname)/Sys.total_memory()

0.5216578024702109

In [41]:
function cortayguarda(filename::String, fmem=0.25)
    

dicth5 = brw_things( filename ); # variables útiles
pointdatos = dicth5[ "dset" ]; # path al dataset (NO READ! to much)
M=size(pointdatos, 1)
    
nframes=dicth5["NRecFrames"]

    println(nframes, "\n", M, "\n", M/4096)
    
nchans=Int(M/nframes)
    
ncachos=min(ceil(Int, 1/fmem), 4) # al menos partela en 4
fcacho=floor(Int, nframes/ncachos) # frames por cacho dara un poco menos que el total.
        
infin = zeros( Int,ι[ 1 ], 2 ); # preallocate
infin[ :, 1 ] = collect( 1:fcacho:nframes ); # inicio y 
infin[ :, 2 ] = infin[ :, 1 ] .+ fcacho .- 1; # fin en frames de cada cacho (para cortar)
#
for i = 1:ncachos  # numero de Β a cortar (1-> 4096,1->ω)
    B = zeros(UInt16, nchans, fcacho ); # preallocate
    # valores correspondientes al Β especifico
    # el canal 1,1 tiene el frame 1, 4097, 8193...etc
    ondeadonde = collect( ( inifin[ i, 1 ] - 1 ):1:inifin[ i, 2 ] ); 
    for j = 1:fcacho
        # saca esos frames del machote seguido Σ, y ponlos en array en Β
        Β[ :, j ] = pointdatos[ ( ondeadonde[ j ]*nchans  + 1):( nchans*ondeadonde[ j + 1 ] ) ];
    end
    #Β = ( ( Β.*ζ ) .+ ο ); # conversion a volaje
    #ΒINname = string( WORKpath, Γ, "Β", add_zeros( 2, i ), ".jld" ); 
    #save( ΒINname, "data", Β );
    #println( add_zeros( 2, i ), " of ", ι[ 1 ], "...saved" );
end
   
    print("bla")
    
end

cortayguarda (generic function with 2 methods)

In [42]:
a["dset"][1:2]

2-element Array{UInt16,1}:
 0x0000
 0x0801

In [43]:
if pmem < 0.25
    println("Tu archivo tiene tamaño razonable. Déjalo así.")
else
    a=brw_things(arxname)
    cortayguarda(arxname)
end

3303268
13164503040
3.21399e6


InexactError: InexactError: Int64(Int64, 3985.2966940617594)

In [44]:
typeof(a["dset"])

HDF5Dataset

In [9]:
arxdict=h5open(arxname)

HDF5 data file: /home/karel/BRWFiles/Isabel2019/control_02.brw

In [10]:
names(arxdict)

3-element Array{String,1}:
 "3BData"    
 "3BRecInfo" 
 "3BUserInfo"

In [14]:
names(arxdict["3BRecInfo/3BRecVars"])

7-element Array{String,1}:
 "BitDepth"       
 "ExperimentType" 
 "MaxVolt"        
 "MinVolt"        
 "NRecFrames"     
 "SamplingRate"   
 "SignalInversion"

In [66]:
b=read(arxdict["/3BData/RawEncodedTOC"])

3×1 Array{Int64,2}:
   89278
 3213990
       0

In [60]:
c=(arxdict["3BRecInfo/3BRecVars/NRecFrames"][1])[1]*4096

13530185728

In [63]:
(c-b)/4096

89278.0

In [69]:
b[1]/18

4959.888888888889

In [67]:
b[2]/b[1]

35.99979838258025

In [68]:
6*18

108